# 🎯 Exercise 1: Build Your First Stateful Agent

**Difficulty:** Beginner  
**Estimated Time:** 30-45 minutes

## Task
Build a simple customer support chatbot that remembers conversation context.

## Requirements
1. Create a StateGraph with MessagesState
2. Add a system prompt that makes the agent act as a helpful customer support rep
3. Use MemorySaver checkpointer for memory
4. Test with a multi-turn conversation where context matters

## Example Conversation
```
User: "I bought a laptop last week"
Agent: "I'd be happy to help with your laptop! What seems to be the issue?"
User: "It won't turn on"
Agent: "I understand your laptop won't turn on. Have you tried..."
```

---

In [1]:
%pip install -q python-dotenv langchain-openai langgraph

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import Annotated, Sequence
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage, SystemMessage
import operator
import os

# ─── Load API key ─────────────────────────────────────────────────────
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded successfully")

# ─── LLM Setup ────────────────────────────────────────────────────────
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

Note: you may need to restart the kernel to use updated packages.
✅ API key loaded successfully
✅ LLM initialized: gpt-4o-mini


## Define the State

We'll use a simple state that stores messages. The `Annotated` with `operator.add` ensures messages accumulate in the conversation history.

In [2]:
from typing import TypedDict

# ─── State Definition ─────────────────────────────────────────────────────
class MessagesState(TypedDict):
    """Simple state that accumulates messages"""
    messages: Annotated[Sequence[BaseMessage], operator.add]

print("✅ State defined: MessagesState")
print(f"   - Stores: conversation messages")
print(f"   - Messages accumulate (operator.add) for conversation history")

✅ State defined: MessagesState
   - Stores: conversation messages
   - Messages accumulate (operator.add) for conversation history


## Create the Agent Node

This is the core node that processes messages and generates responses.

In [3]:
# ─── Agent Node ────────────────────────────────────────────────────────────
def customer_support_agent(state: MessagesState):
    """
    Main agent node. Processes the current messages and generates a response.
    The system prompt defines the agent's personality and behavior.
    """
    system_prompt = """You are a friendly and helpful customer support representative.

Your responsibilities:
- Remember what the customer has told you in the conversation history
- Address their concerns with empathy and professionalism
- Ask clarifying questions if needed
- Provide helpful solutions or next steps
- Be concise but thorough in your responses

Important: Use the conversation context to provide personalized, relevant responses.
For example, if they mention a product, remember it in subsequent messages."""

    # Prepend system message to the conversation history
    messages = [SystemMessage(content=system_prompt)] + list(state["messages"])
    
    # Call the LLM with all messages (history + new message)
    response = llm.invoke(messages)
    
    # Return the response to be added to messages
    return {"messages": [response]}

print("✅ Agent node created: customer_support_agent")
print("   - Uses system prompt to define customer support behavior")
print("   - Receives conversation history automatically")

✅ Agent node created: customer_support_agent
   - Uses system prompt to define customer support behavior
   - Receives conversation history automatically


## Build the Graph

Connect the node with START and END edges to create the graph.

In [4]:
# ─── Build StateGraph ──────────────────────────────────────────────────────
builder = StateGraph(MessagesState)

# Add the agent node
builder.add_node("agent", customer_support_agent)

# Connect START → agent → END
builder.add_edge(START, "agent")
builder.add_edge("agent", END)

# Compile with MemorySaver for conversation persistence
checkpointer = MemorySaver()
chatbot = builder.compile(checkpointer=checkpointer)

print("✅ Graph built and compiled with MemorySaver checkpointer")
print("   - Nodes: START → agent → END")
print("   - Memory: Enabled (MemorySaver)")
print("   - Each conversation gets a thread_id for separate memory")

✅ Graph built and compiled with MemorySaver checkpointer
   - Nodes: START → agent → END
   - Memory: Enabled (MemorySaver)
   - Each conversation gets a thread_id for separate memory


## Test the Chatbot

Test with a multi-turn conversation where context matters.

In [5]:
# ─── Test Function ────────────────────────────────────────────────────────
def chat_with_support(user_message: str, thread_id: str = "customer_001"):
    """
    Send a message to the support chatbot and get a response.
    The thread_id ensures conversation history is maintained.
    """
    print(f"\n👤 Customer: {user_message}")
    
    # Invoke the graph with the new message
    result = chatbot.invoke(
        {"messages": [HumanMessage(content=user_message)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    # Extract and print the agent's response (last message)
    agent_response = result["messages"][-1]
    print(f"🤖 Agent: {agent_response.content}")
    
    return result

print("✅ Test function created: chat_with_support()")

✅ Test function created: chat_with_support()


## Multi-Turn Conversation

Test that the agent remembers context across turns.

In [6]:
print("\n" + "═"*80)
print("TEST 1: Multi-Turn Conversation (Agent Remembers Context)")
print("═"*80)

# Turn 1
chat_with_support(
    "I bought a laptop last week",
    thread_id="customer_001"
)

# Turn 2 - Agent should remember it's about a laptop
chat_with_support(
    "It won't turn on",
    thread_id="customer_001"
)

# Turn 3 - Agent should remember both facts
chat_with_support(
    "I've tried holding the power button for 30 seconds",
    thread_id="customer_001"
)


════════════════════════════════════════════════════════════════════════════════
TEST 1: Multi-Turn Conversation (Agent Remembers Context)
════════════════════════════════════════════════════════════════════════════════

👤 Customer: I bought a laptop last week
🤖 Agent: That's great to hear! How is your new laptop working for you so far? If you have any questions or need assistance with it, feel free to ask!

👤 Customer: It won't turn on
🤖 Agent: I'm sorry to hear that your laptop won't turn on. That can be really frustrating. Here are a few steps you can try to troubleshoot the issue:

1. **Check the Power Supply**: Make sure the laptop is plugged in and the power outlet is working. If you have a different charger, you might want to try using that.

2. **Remove External Devices**: Disconnect any peripherals (like USB drives or external monitors) and try turning it on again.

3. **Perform a Hard Reset**: If your laptop has a removable battery, take it out, unplug the power adapter, and

{'messages': [HumanMessage(content='I bought a laptop last week', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's great to hear! How is your new laptop working for you so far? If you have any questions or need assistance with it, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 105, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CyeWV1jxj5Loq5YfvcWl0hYXfzX0b', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bc713-3dd9-7fc0-b5b7-c92f3164c6e3-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 105, 'output_tokens': 32, 'tot

## Different Customer (Different Thread)

Show that different thread_ids maintain separate conversations.

In [7]:
print("\n" + "═"*80)
print("TEST 2: Different Customer (Separate Thread - No Context Bleed)")
print("═"*80)

# Different thread_id = different conversation
chat_with_support(
    "Hi, I have a problem with my phone order",
    thread_id="customer_002"
)

chat_with_support(
    "The screen is cracked",
    thread_id="customer_002"
)

print("\n💡 Notice: Customer 2's conversation is independent of Customer 1's")


════════════════════════════════════════════════════════════════════════════════
TEST 2: Different Customer (Separate Thread - No Context Bleed)
════════════════════════════════════════════════════════════════════════════════

👤 Customer: Hi, I have a problem with my phone order
🤖 Agent: I'm sorry to hear that you're having a problem with your phone order. Could you please provide me with more details about the issue? This will help me assist you better!

👤 Customer: The screen is cracked
🤖 Agent: I'm really sorry to hear that your phone arrived with a cracked screen. That must be frustrating. Could you let me know if you received the phone recently and if it was damaged during shipping? Additionally, do you have any order details or pictures of the damage that you could share? This will help me find the best solution for you.

💡 Notice: Customer 2's conversation is independent of Customer 1's


## Verify Memory Persistence

After several turns, verify the agent has full conversation history.

In [8]:
print("\n" + "═"*80)
print("TEST 3: Check Memory Persistence")
print("═"*80)

# Get the state snapshot to see full conversation history
state_snapshot = chatbot.get_state(config={"configurable": {"thread_id": "customer_001"}})

print("\nFull Conversation History for Customer 001:")
print("─" * 80)

for i, msg in enumerate(state_snapshot.values["messages"], 1):
    if isinstance(msg, HumanMessage):
        print(f"{i}. 👤 Customer: {msg.content}")
    elif isinstance(msg, AIMessage):
        print(f"{i}. 🤖 Agent: {msg.content[:100]}..." if len(msg.content) > 100 else f"{i}. 🤖 Agent: {msg.content}")
    print()

print(f"\n✅ Total messages stored: {len(state_snapshot.values['messages'])}")
print(f"✅ Memory is working! The agent has full context across turns.")


════════════════════════════════════════════════════════════════════════════════
TEST 3: Check Memory Persistence
════════════════════════════════════════════════════════════════════════════════

Full Conversation History for Customer 001:
────────────────────────────────────────────────────────────────────────────────
1. 👤 Customer: I bought a laptop last week

2. 🤖 Agent: That's great to hear! How is your new laptop working for you so far? If you have any questions or ne...

3. 👤 Customer: It won't turn on

4. 🤖 Agent: I'm sorry to hear that your laptop won't turn on. That can be really frustrating. Here are a few ste...

5. 👤 Customer: I've tried holding the power button for 30 seconds

6. 🤖 Agent: Thank you for sharing that. Since you've already tried holding the power button for 30 seconds and i...


✅ Total messages stored: 6
✅ Memory is working! The agent has full context across turns.
